In [1]:
from shapely.geometry import Point
from collections import Counter
import seaborn as sns
import datetime as datetime
from datetime import date
import io
import re
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import folium
from folium.plugins import MarkerCluster
from folium.plugins import FastMarkerCluster
import matplotlib.pyplot as plt
import numpy as np

%matplotlib inline

In [2]:
mma_data = pd.read_csv('UFC.csv')

/opt/anaconda3/envs/geospatial/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3441: DtypeWarning: Columns (144,145) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
mma_data

,R_fighter,B_fighter,Referee,date,location,Winner,title_bout,weight_class,B_avg_KD,B_avg_opp_KD,...,R_Stance,R_Height_cms,R_Reach_cms,R_Weight_lbs,B_age,R_age,R_index_ape,B_index_ape,R_index_ape2,B_index_ape2
0,Adrian Yanez,Gustavo Lopez,Chris Tognoni,3/20/2021,"Las Vegas, Nevada, USA",Red,False,Bantamweight,0.000,0.0,...,Orthodox,170.18,177.80,135.0,31.0,27.0,1.04,1.03,7.62,5.08
1,Trevin Giles,Roman Dolidze,Herb Dean,3/20/2021,"Las Vegas, Nevada, USA",Red,False,Middleweight,0.500,0.0,...,Orthodox,182.88,187.96,185.0,32.0,28.0,1.03,1.03,5.08,5.08
2,Tai Tuivasa,Harry Hunsucker,Herb Dean,3/20/2021,"Las Vegas, Nevada, USA",Red,False,Heavyweight,NaN,NaN,...,Southpaw,187.96,190.50,264.0,32.0,28.0,1.01,1.01,2.54,2.54
3,Cheyanne Buys,Montserrat Conejo,Mark Smith,3/20/2021,"Las Vegas, Nevada, USA",Blue,False,WomenStrawweight,NaN,NaN,...,Switch,160.02,160.02,115.0,28.0,25.0,1.0,1.02,0.00,2.54
4,Marion Reneau,Macy Chiasson,Mark Smith,3/20/2021,"Las Vegas, Nevada, USA",Blue,False,WomenBantamweight,0.125,0.0,...,Orthodox,167.64,172.72,135.0,29.0,43.0,1.03,1.01,5.08,2.54
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6007,Remco Pardoel,Alberta Cerra Leon,John McCarthy,3/11/1994,"Denver, Colorado, USA",Red,False,OpenWeight,NaN,NaN,...,Southpaw,190.50,NaN,260.0,NaN,NaN,0,0,-190.50,-172.72
6008,Orlando Wiet,Robert Lucarelli,John McCarthy,3/11/1994,"Denver, Colorado, USA",Red,False,OpenWeight,NaN,NaN,...,Southpaw,177.80,NaN,170.0,NaN,NaN,0,0,-177.80,-187.96
6009,Johnny Rhodes,David Levicki,John McCarthy,3/11/1994,"Denver, Colorado, USA",Red,False,OpenWeight,NaN,NaN,...,Orthodox,182.88,NaN,210.0,NaN,NaN,0,0,-182.88,-195.58
6010,Patrick Smith,Ray Wizard,John McCarthy,3/11/1994,"Denver, Colorado, USA",Red,False,OpenWeight,NaN,NaN,...,Orthodox,187.96,NaN,225.0,NaN,30.0,0,#DIV/0!,-187.96,0.00


In [4]:
mma_data[mma_data['R_fighter'] == 'Dustin Poirier']

,R_fighter,B_fighter,Referee,date,location,Winner,title_bout,weight_class,B_avg_KD,B_avg_opp_KD,...,R_Stance,R_Height_cms,R_Reach_cms,R_Weight_lbs,B_age,R_age,R_index_ape,B_index_ape,R_index_ape2,B_index_ape2
85,Dustin Poirier,Conor McGregor,Herb Dean,1/23/2021,"Abu Dhabi, Abu Dhabi, United Arab Emirates",Red,False,Lightweight,1.093262,0.000000,...,Southpaw,175.26,182.88,155.0,32.0,32.0,1.04,1.07,7.62,12.70
395,Dustin Poirier,Dan Hooker,Herb Dean,6/27/2020,"Las Vegas, Nevada, USA",Red,False,Lightweight,0.457275,0.062500,...,Southpaw,175.26,182.88,155.0,30.0,31.0,1.04,1.04,7.62,7.62
1437,Dustin Poirier,Justin Gaethje,Herb Dean,4/14/2018,"Glendale, Arizona, USA",Red,False,Lightweight,0.000000,0.500000,...,Southpaw,175.26,182.88,155.0,29.0,29.0,1.04,0.99,7.62,-2.54
1650,Dustin Poirier,Anthony Pettis,Keith Peterson,11/11/2017,"Norfolk, Virginia, USA",Red,False,Lightweight,0.002930,0.250000,...,Southpaw,175.26,182.88,155.0,30.0,28.0,1.04,1.03,7.62,5.08
1976,Dustin Poirier,Jim Miller,Todd Ronald Anderson,2/11/2017,"Brooklyn, New York, USA",Red,False,Lightweight,0.000017,0.003906,...,Southpaw,175.26,182.88,155.0,33.0,28.0,1.04,1.04,7.62,7.62
2176,Dustin Poirier,Michael Johnson,Dan Miragliotta,9/17/2016,"Hidalgo, Texas, USA",Blue,False,Lightweight,0.073486,0.001953,...,Southpaw,175.26,182.88,155.0,30.0,27.0,1.04,1.04,7.62,7.62
2333,Dustin Poirier,Bobby Green,Jason Herzog,6/4/2016,"Los Angeles, California, USA",Red,False,Lightweight,0.062500,0.000000,...,Southpaw,175.26,182.88,155.0,29.0,27.0,1.04,1.01,7.62,2.54
2509,Dustin Poirier,Joe Duffy,Chris Tognoni,1/2/2016,"Las Vegas, Nevada, USA",Red,False,Lightweight,0.500000,0.000000,...,Southpaw,175.26,182.88,155.0,27.0,26.0,1.04,1.04,7.62,7.62
2783,Dustin Poirier,Yancy Medeiros,John McCarthy,6/6/2015,"New Orleans, Louisiana, USA",Red,False,Lightweight,0.562500,0.000000,...,Southpaw,175.26,182.88,155.0,27.0,26.0,1.04,1.07,7.62,12.70
2879,Dustin Poirier,Diego Ferreira,Todd McGovern,4/4/2015,"Fairfax, Virginia, USA",Red,False,Lightweight,0.250000,0.000000,...,Southpaw,175.26,182.88,155.0,30.0,26.0,1.04,1.07,7.62,12.70


In [5]:
mma_data[mma_data['B_fighter'] == 'Dustin Poirier']

,R_fighter,B_fighter,Referee,date,location,Winner,title_bout,weight_class,B_avg_KD,B_avg_opp_KD,...,R_Stance,R_Height_cms,R_Reach_cms,R_Weight_lbs,B_age,R_age,R_index_ape,B_index_ape,R_index_ape2,B_index_ape2
737,Khabib Nurmagomedov,Dustin Poirier,Marc Goddard,9/7/2019,"Abu Dhabi, Abu Dhabi, United Arab Emirates",Red,True,Lightweight,0.260986,0.008057,...,Orthodox,177.80,177.80,155.0,30.0,30.0,1.0,1.04,0.00,7.62
951,Max Holloway,Dustin Poirier,Blake Grice,4/13/2019,"Atlanta, Georgia, USA",Blue,True,Lightweight,0.521973,0.016113,...,Orthodox,180.34,175.26,145.0,30.0,27.0,0.97,1.04,-5.08,7.62
1292,Eddie Alvarez,Dustin Poirier,Marc Goddard,7/28/2018,"Calgary, Alberta, Canada",Blue,False,Lightweight,0.043945,0.032227,...,Orthodox,175.26,175.26,155.0,29.0,34.0,1.0,1.04,0.00,7.62
1880,Eddie Alvarez,Dustin Poirier,Herb Dean,5/13/2017,"Dallas, Texas, USA",Draw,False,Lightweight,0.351562,0.257812,...,Orthodox,175.26,175.26,155.0,28.0,33.0,1.0,1.04,0.00,7.62
3629,Erik Koch,Dustin Poirier,Rob Hinds,8/31/2013,"Milwaukee, Wisconsin, USA",Blue,False,Featherweight,0.000000,0.000000,...,Southpaw,177.80,180.34,170.0,24.0,24.0,1.01,1.04,2.54,7.62
3829,Cub Swanson,Dustin Poirier,Kevin Mulhall,2/16/2013,"London, England, United Kingdom",Red,False,Featherweight,0.000000,0.000000,...,Orthodox,172.72,177.80,145.0,24.0,29.0,1.03,1.04,5.08,7.62
4101,Chan Sung Jung,Dustin Poirier,Dan Miragliotta,5/15/2012,"Fairfax, Virginia, USA",Red,False,Featherweight,0.000000,0.000000,...,Orthodox,170.18,182.88,145.0,23.0,25.0,1.07,1.04,12.70,7.62
